In [1]:
import torch
import torchvision
from torchvision import transforms
import torch.utils.data as data
import torch.optim as optim
import torch.nn as nn
from torchsummary import summary
import time
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms,models
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from pathlib import Path
import pandas as pd
import cv2

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [2]:
device = torch.device('cuda')
device=torch.device(device)

In [3]:
import gc

gc.collect()

torch.cuda.empty_cache()




In [4]:

BATCH_SIZE = 64
learning_rate = 0.0001
EPOCHS = 5
numClasses = 205

In [5]:
# def collate_fn(batch):
#     return tuple(zip(*batch))


In [6]:
class TrafficData(Dataset):
    def __init__(self, df, image_dir, transforms=None):
        self.image_ids = df['Path'].unique()
        self.df = df
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index, size = [112, 112]):
        image_path = self.image_ids[index]
        records = self.df[self.df['Path'] == image_path]

        #print(f'{self.image_dir}/{image_path}')
        image = cv2.imread(f'{self.image_dir}/{image_path}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, size)
        image = image.astype(float) / 255.0

        target = records['ClassId'].values

        if self.transforms:
            image = self.transforms(**image)

        return image, target

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    @staticmethod
    def create_dataset(df, dir, transform=None):
       dataset = TrafficData(df, dir)
       return dataset

    @staticmethod 
    def loader(dataset, batch_size, num_workers=0):
       data_loader = DataLoader(
          dataset,
          batch_size=batch_size,
          shuffle=True,
          num_workers=num_workers,
#           collate_fn = collate_fn
      )
       return data_loader


In [7]:
path = Path("Data_images")


In [8]:
df_train = pd.read_csv("Data_Images/Train_balanced_data.csv")
df_test = pd.read_csv("Data_images/Test_data_cleaned.csv")


In [9]:

data_transforms = transforms.Compose([
    transforms.Resize([112, 112]),
    transforms.ToTensor()
    ])


In [10]:
train_data = TrafficData.create_dataset(df_train, path, data_transforms)
train_data_loader = TrafficData.loader(train_data, BATCH_SIZE)

test_data = TrafficData.create_dataset(df_test, path, data_transforms)
test_data_loader = TrafficData.loader(test_data, BATCH_SIZE)


In [11]:
images, targets = next(iter(test_data_loader))

In [12]:
for X, y in test_data_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 112, 112, 3])
Shape of y: torch.Size([64, 1]) torch.int64


In [13]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [14]:
eff_net_b0 = models.efficientnet_b0(pretrained=True)

class Eff_net(nn.Module):
    def __init__(self, pretrained_model):
        super(Eff_net,self).__init__()
        self.Eff_net = pretrained_model
        #num_inputs = pretrained_model.classifier[1].in_features
        self.fl1 = nn.Linear(1000, BATCH_SIZE)
        self.fl2 = nn.Linear(BATCH_SIZE, numClasses)
        
    def forward(self, X):
        X = self.Eff_net(X)
        X = F.relu(self.fl1(X))
        X = F.dropout(X, p=0.25)
        X = self.fl2(X)
        return X


E:\trafic_sign_classification\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
E:\trafic_sign_classification\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
model = Eff_net(eff_net_b0)

In [16]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [17]:
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [18]:

def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [19]:
def train(model, loader, opt, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    
    for (images, labels) in loader:
        images = images.permute(0,3,1,2)
        images = images.float()
        labels = torch.reshape(labels, (-1,))
        images = images.cuda()
        labels = labels.cuda()
        
        opt.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        

        loss.backward()
        
        acc = calculate_accuracy(output, labels)
        
        opt.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)


In [20]:
def evaluate(model, loader, opt, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for (images, labels) in loader:
            images = images.permute(0,3,1,2)
            images = images.float()
            labels = torch.reshape(labels, (-1,))
            images = images.cuda()
            labels = labels.cuda()
            
            output = model(images)
            loss = criterion(output, labels)
            
            acc = calculate_accuracy(output, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [21]:
train_loss_list = [0]*EPOCHS
train_acc_list = [0]*EPOCHS
val_loss_list = [0]*EPOCHS
val_acc_list = [0]*EPOCHS

for epoch in range(EPOCHS):
    print("Epoch-%d: " % (epoch))

    train_start_time = time.monotonic()
    train_loss, train_acc = train(model, train_data_loader, optimizer, criterion)
    train_end_time = time.monotonic()

    val_start_time = time.monotonic()
    val_loss, val_acc = evaluate(model, test_data_loader, optimizer, criterion)
    val_end_time = time.monotonic()
    
    train_loss_list[epoch] = train_loss
    train_acc_list[epoch] = train_acc
    val_loss_list[epoch] = val_loss
    val_acc_list[epoch] = val_acc
    
    print("Training: Loss = %.4f, Accuracy = %.4f, Time = %.2f seconds" % (train_loss, train_acc, train_end_time - train_start_time))
    print("Validation: Loss = %.4f, Accuracy = %.4f, Time = %.2f seconds" % (val_loss, val_acc, val_end_time - val_start_time))
    print("")

Epoch-0: 
Training: Loss = 3.1380, Accuracy = 0.3614, Time = 1013.19 seconds
Validation: Loss = 0.8786, Accuracy = 0.7945, Time = 380.50 seconds

Epoch-1: 
Training: Loss = 0.4853, Accuracy = 0.8786, Time = 1050.12 seconds
Validation: Loss = 0.3241, Accuracy = 0.9217, Time = 385.19 seconds

Epoch-2: 
Training: Loss = 0.1823, Accuracy = 0.9544, Time = 1021.64 seconds
Validation: Loss = 0.2098, Accuracy = 0.9507, Time = 356.69 seconds

Epoch-3: 
Training: Loss = 0.0993, Accuracy = 0.9757, Time = 1043.61 seconds
Validation: Loss = 0.1806, Accuracy = 0.9585, Time = 372.62 seconds

Epoch-4: 
Training: Loss = 0.0666, Accuracy = 0.9837, Time = 1013.22 seconds
Validation: Loss = 0.1549, Accuracy = 0.9664, Time = 389.84 seconds

